In [2]:
import re
import numpy as np

In [7]:
a = [[1,5],[2,3]]
np.size(a)

4

In [8]:
# match_RE = r"([+-]?(?:\(?\d+\.?\d*\)?)(?:[eE][+-]?\d+)?)|\(([^)]+)\)"
pattern = r'([+-]?(?:\d+\.?\d*|\.\d+)(?:[eE][+-]?\d+)?)|\(([^)]+)\)'
text = "1.4,71.97820422200603,-71.97820422200601,(0.968603747872373+0j),(-0.24860969330979957-6.757115257174406e-15j),(0.24294965064438834-4.121356615000312e-15j),(0.9700388998652423+0j)"
re.findall(pattern, text)

[('1.4', ''),
 ('71.97820422200603', ''),
 ('-71.97820422200601', ''),
 ('', '0.968603747872373+0j'),
 ('', '-0.24860969330979957-6.757115257174406e-15j'),
 ('', '0.24294965064438834-4.121356615000312e-15j'),
 ('', '0.9700388998652423+0j')]

In [5]:
import re

# 输入的字符串
s = "1540,1,1.470896388852969+2.253110692262893e-15i,1.589330270267854-2.795949688778033e-13i,7.984651821069383e-08,0.9997626464955753,6021.647459995122"

int_re      = r'\d+'
complex_re  = r'[-+]?(?:\d+\.?\d*|\.\d+)(?:[eE][-+]?\d+)?[+-](?:\d+\.?\d*|\.\d+)(?:[eE][-+]?\d+)?i'
float_re    = r'[-+]?(?:\d+\.?\d*|\.\d+)(?:[eE][-+]?\d+)?'
pattern     = rf'^{int_re},{int_re},{complex_re},{complex_re},{float_re},{float_re},{float_re}$'

# pattern = r'|([+-]?(?:\d+\.?\d*)(?:[eE][+-]?\d+)?)|\(([^)]+)\)|([+-]?\d+)'
matches = re.findall(pattern, s)
matches
# values = []
# for integer, real, complex_num in matches:
#     if integer:
#         values.append(int(integer))
#     elif real:
#         values.append(float(real))
#     else:
#         values.append(complex(complex_num))

['1540,1,1.470896388852969+2.253110692262893e-15i,1.589330270267854-2.795949688778033e-13i,7.984651821069383e-08,0.9997626464955753,6021.647459995122']